In [13]:
import pandas as pd
import pickle
from datetime import datetime

# Function to predict return
def predict_return(stock_symbol, date, open_price, volume, weighted_sentiment_score):
    """
    Predict return based on the inputs using the saved SARIMAX model.

    Args:
        stock_symbol (str): The stock symbol (e.g., 'AAPL', 'TSLA').
        date (str): The date for prediction (format: 'YYYY-MM-DD').
        open_price (float): The open price of the stock.
        volume (int): The volume of the stock.
        sentiment_score (int): Sentiment score (-1, 0, or 1).
        confidence_interval (float): Confidence interval for the sentiment score.

    Returns:
        float: Predicted return for the given inputs.
    """
    # Calculate weighted sentiment score

    # Load the saved SARIMAX model for the stock
    model_path = f'/Users/wenjing/Downloads/Capstone/saved_models/{stock_symbol}_sarimax_model.pkl'
    try:
        with open(model_path, 'rb') as model_file:
            model = pickle.load(model_file)
    except FileNotFoundError:
        raise ValueError(f"Model for stock '{stock_symbol}' not found at {model_path}.")

    # Prepare exogenous variables as a DataFrame
    exog = pd.DataFrame({
        'Date': [datetime.strptime(date, '%Y-%m-%d')],
        'open': [open_price],
        'volume': [volume],
        'weighted_sentiment_score': [weighted_sentiment_score]
    })

    # Predict the return using the loaded SARIMAX model
    predictions = model.get_forecast(steps=1, exog=exog[['open', 'volume', 'weighted_sentiment_score']]).predicted_mean

    # Extract the first prediction
    prediction = predictions.iloc[0]

    return prediction


In [14]:
# Example inputs
stock_symbol = 'AAPL'
date = '2024-12-16'
open_price = 198
volume = 88570300

weighted_sentiment_score = 2
# Predict the return
predicted_return = predict_return(stock_symbol, date, open_price, volume, weighted_sentiment_score)

print(f"Predicted return for {stock_symbol} on {date}: {predicted_return}")


Predicted return for AAPL on 2024-12-16: 0.004381346815141106


In [16]:

# Define the prediction endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    stock_symbol = data.get('stock_symbol')
    date = data.get('date')
    open_price = data.get('open_price')
    volume = data.get('volume')
    weighted_sentiment_score = data.get('weighted_sentiment_score')

    # Validate inputs
    if not stock_symbol or not date or open_price is None or volume is None or weighted_sentiment_score is None:
        return jsonify({'error': 'Invalid input. Please provide all required fields.'}), 400

    try:
        # Call the prediction function
        predicted_return = predict_return(
            stock_symbol, date, open_price, volume, weighted_sentiment_score
        )
        return jsonify({'stock_symbol': stock_symbol, 'predicted_return': predicted_return})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

AssertionError: View function mapping is overwriting an existing endpoint function: predict